# This Notebook provides the walk through each step for single sleep-EEG (".edf") file to present some details about possible the package in details - Part3.

The whole notebooks parts should be run together inorder to explain the first each note-books are provided seperately.
This will help the user to identify the any-issues while first time running the package.

However these parts include the first part and wanted functions in the first block.
So the user can run the first block as whole to finish the previous parts and check the next stages.

If the user want to run the whole script together pleace check the last-note-book.

## Upto part-2 and needed modules

Since you are using the script to run the file, please load the latest .whl file 
located in the "EEG_PSG_loose_lead_test/dist/sleep_EEG_loose_lead_detect-0.0-py3-none-any.whl"

Once the package is dowloaded, then change the directory via the command in cmd-prompt

cd .../EEG_PSG_loose_lead_test/dist/

Then install the packge via the pip command via the command in cmd-prompt,

pip install sleep_EEG_loose_lead_detect-0.0-py3-none-any.whl

In [ ]:
import os
import sys
import logging

import numpy as np
import pandas as pd
import pickle


from copy import deepcopy
from collections import Counter
# from multiprocessing import Pool#, TimeoutError

import sleep_EEG_loose_lead_detect as sleep_EEG_loose_lead_detect
# --------------------------------------------------------------------------
# load package functions
# assign the working directory of the package
# --------------------------------------------------------------------------
from sleep_EEG_loose_lead_detect.directory_utils import EEG_sleep_dir

from sleep_EEG_loose_lead_detect.optional_parameters import parameter_assignment

#checking only one get_root_channels function whether the loading suceeds
# # here non-normlaised EEG is obtained in the preproceeing step in time domain
from sleep_EEG_loose_lead_detect.preprocessing.load_EDF_dataset_events import load_root_dataset
from sleep_EEG_loose_lead_detect.preprocessing.find_bad_epochs import markBadEpochs
from sleep_EEG_loose_lead_detect.preprocessing.segment_filter_EEG import segment_EEG
from sleep_EEG_loose_lead_detect.preprocessing.cont_segs_of_whole_EEG import save_cont_segs, continious_seg_to_np
# --------------------------------------------------------------------------
# spindle retaleed function based on YASA
# --------------------------------------------------------------------------
from sleep_EEG_loose_lead_detect.preprocessing.spindle_fun import spindle_detect_main
from sleep_EEG_loose_lead_detect.preprocessing.detected_spindle_one_hot import convert_to_full_one_hot_mapping_based_cont_seg_sp_sw_all_channels


from sleep_EEG_loose_lead_detect.loose_lead_events.events_to_txt import preprocess_events_to_txt

from sleep_EEG_loose_lead_detect.channel_correlation_outlier.correlation_functions import MT_based_correltion_calc_in_continious_segs


logging.basicConfig(level=logging.INFO)

logger = logging.getLogger("sleep_loose_lead")
while logger.handlers:
      logger.handlers.pop()
c_handler = logging.StreamHandler()
# link handler to logger
logger.addHandler(c_handler)
logger.setLevel(logging.INFO)
logger.propagate = False



'''
direct_run makes skip the user inputs
'''
loading_dir_pre =EEG_sleep_dir(splidle_inc=False)
# --------------------------------------------------------------------------
#  first assign the i/p and o/p directories
# --------------------------------------------------------------------------
loading_dir_pre.in_loc = '/Users/anandanadarajn/Documents/.../results_pickles/check_loose_lead/'
loading_dir_pre.out_loc ='/Users/anandanadarajn/Documents/.../results_pickles/check_loose_lead/'

loading_dir_pre.keep_signature_dic ={}
loading_dir_pre.keep_signature_dic['dic']=True
loading_dir_pre.keep_signature_dic['evtxt']=True
loading_dir_pre.keep_signature_dic['bad_epochs']=True
loading_dir_pre.keep_signature_dic['out_loc_outlier']=True
loading_dir_pre.keep_signature_dic['sleep_anot']=True
loading_dir_pre.keep_signature_dic['MT_spec']=True
loading_dir_pre.keep_signature_dic['annota_NREM_REM']=True
loading_dir_pre.keep_signature_dic['splidle_inc']=True
loading_dir_pre.keep_signature_dic['tex_files']=True

loading_dir_pre.save_spindle_loc = True

# --------------------------------------------------------------------------
# re assign the loading_dir_pre object
# --------------------------------------------------------------------------
loading_dir_pre.assign_directories()
f='edf_name'

'''
getting user input and call loose-lead detection
'''
opt_paramters = parameter_assignment()
opt_paramters.loading_dir_pre = loading_dir_pre

opt_paramters.sep = False

opt_paramters.tag='_def'

pred_slow_waves = opt_paramters.pred_slow_waves
pred_spindles = opt_paramters.pred_spindles 

T = opt_paramters.T
amplitude_high_same_all_age = opt_paramters.amplitude_high_same_all_age
avoid_spindle_loc = opt_paramters.avoid_spindle_loc
verbose  = opt_paramters.verbose


# --------------------------------------------------------------------------
# assigning the directories
# --------------------------------------------------------------------------
in_loc = loading_dir_pre.in_loc
# save_converted_index_loc = loading_dir_pre.save_converted_index_loc
# out_loc_outlier = loading_dir_pre.out_loc_outlier
in_bad_events = loading_dir_pre.bad_events_file_path
save_spindle_loc = loading_dir_pre.save_spindle_loc


# --------------------------------------------------------------------------
#  if the file not ends with edf add edf extention
# --------------------------------------------------------------------------

if in_name_temp.split('.')[-1]!='edf':
    in_edf = in_loc + in_name_temp + ".edf"
else:
    in_edf = in_loc + in_name_temp 
    in_name_temp = '.'.join(in_name_temp.split('.')[0:-1])
logger.warning('*** filename: %s', in_edf)

if len(opt_paramters.tag)>0:
    in_name_temp= in_name_temp+opt_paramters.tag

epoch_length, line_freq, bandpass_freq, normal_only, notch_freq_essential_checker, amplitude_thres = opt_paramters.preprocess_par()

amplitude_thres =2000
# --------------------------------------------------------------------------
# developments optional to user
# --------------------------------------------------------------------------
save_events_origin = opt_paramters.save_events_origin
sleep_stage_preprocess_origin = opt_paramters.sleep_stage_preprocess_origin

os.chdir('/')#to make sure appending paths
EEG_root, sleep_stages_or, EEG_channels, Fs, start_time_idx,  sel_index_sl_st, whole_annotations, ev_or = load_root_dataset(in_edf, epoch_length)

# --------------------------------------------------------------------------
# save the events for checking purpose
# --------------------------------------------------------------------------
if save_events_origin:
    np.save(loading_dir_pre.out_loc_NREM_REM+ in_name_temp + "_sel_index_sl_st",sel_index_sl_st)
    np.save(loading_dir_pre.out_loc_NREM_REM+ in_name_temp + "_ev_or",ev_or)
    
bad_epochs = markBadEpochs(in_edf, in_bad_events, epoch_sec=epoch_length)
if loading_dir_pre.keep_signature_dic['bad_epochs']:
    np.save(loading_dir_pre.bad_epochs_folder+ in_name_temp + "_bad_epochs",bad_epochs)
#  make deep copy 
EEG_root_copy=deepcopy(EEG_root)
if avoid_spindle_loc:
    sleep_stages_origin =  deepcopy(sleep_stages_or)


sleep_stages=  deepcopy(sleep_stages_or)
# --------------------------------------------------------------------------
# Segment EEG into 30sec epochs, apply notch & band filters, mark bad epochs and normalization
# this is preprocessing the EEG signal in time domain and return the preprocessed signal in time domain
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# this will force the events with the channels information
#   channel_specific_preprocess=True 
# if not channel_specific_preprocess just return the epoch status without channel specific information
# like nan value, high/lower amplitude etc.
# --------------------------------------------------------------------------
channel_specific_preprocess = opt_paramters.channel_specific_preprocess
# --------------------------------------------------------------------------
# EEG_channels extracted from the load_root_dataset
# such that filaly the default channels will be endup in
# ch_names = ['F3', 'F4', 'C3', 'C4', 'O1', 'O2']
# --------------------------------------------------------------------------
ch_names = EEG_channels    

# --------------------------------------------------------------------------
# in the fuiltertaion process the 
# EEG_root_copy have higher chance of filetisation
#  so keep them in a seperate deep copy is prefered
# 
#  to feed the filterd EEG through the MT-make sure assign this as True
# basically the epoches points to the EEG_root_copies location
# --------------------------------------------------------------------------
filtered_EEG_MT=True
epochs, EEG_root_copy, sleep_stages, epoch_start_idx_o, epoch_status, q1,q2,q3, notch_filter_skipped = segment_EEG(EEG_root_copy, 
                                                                                sleep_stages, 
                                                                                epoch_length, 
                                                                                Fs, start_time_idx, 
                                                                                notch_freq=line_freq, 
                                                                                bandpass_freq=bandpass_freq, 
                                                                                amplitude_thres=amplitude_thres, 
                                                                                channel_specific_preprocess=channel_specific_preprocess,
                                                                                ch_names=ch_names,
                                                                                bad_epochs=bad_epochs,notch_freq_essential_checker=notch_freq_essential_checker,
                                                                                return_filtered_EEG=True,
                                                                                verbose=verbose)

preprocess_events_to_txt(in_name_temp+'_pre_process.txt',loading_dir_pre.out_loc_txt, epoch_status,  preprocess_txt_only_detected_bad=True)
temp= {}
temp['Fs']=Fs

temp['q1']=np.array(q1)
temp['q2']=np.array(q2)
temp['q3']=np.array(q3)

temp['notch_filter_skipped']=notch_filter_skipped

temp['empty_EEG_segs']=False
if epochs.shape[0] <= 0:
    temp['empty_EEG_segs']=True
    raise ValueError('Empty EEG segments')

if normal_only:
    good_ids = np.where(epoch_status=='normal')[0]
    sel_id_name='good_ids'
    if len(good_ids)<=300:
        temp['less_300']=True
    else:
        temp['less_300']=False
else:
    raise Exception("need to modify the epoch status to selecting status, sin cethe outlier detection is build based only on normal epoch in mind")

Here we pass the good_ids (annoated as "normal" by preprocessing). 
Then used by spindle_detect_main to extract the spindle/ slow-wave information.
Or directly obtain only the continious segmentations.

In [ ]:
# --------------------------------------------------------------------------
#for the time being only the good ids we can separatly chechk the higher/ lower cropped portions
# --------------------------------------------------------------------------
sel_ids = good_ids

# --------------------------------------------------------------------------
#  When we are avoiding the spindle occuring location
#   while creating the spindle location the connious segmentation is done
# 
# --------------------------------------------------------------------------
if avoid_spindle_loc:
    # --------------------------------------------------------------------------
    #  for spindle detect ion we feed the unpreprocessed (not filter applied) the EEG data
    # --------------------------------------------------------------------------
    data=deepcopy(EEG_root)

    # spindle detection intiated 
    sw_columns_ordered =  ['Start', 'End', 'MidCrossing','Duration', 'NegPeak', 'PosPeak',  'ValNegPeak', 'ValPosPeak', 
                'PTP', 'Slope', 'Frequency',  'IdxChannel']

    sp_columns_ordered =  ['Start', 'End', 'Duration', 'Peak', 'Amplitude', 'RMS', 'AbsPower',
            'RelPower', 'Frequency', 'Oscillations', 'Symmetry', 'IdxChannel']
    
    pred_slow_waves=True
    pred_spindles =True
    
    cont_EEG_segments,start_keys,cont_EEG_segments_np, sp_sw_dic_format =  spindle_detect_main(in_name_temp, sel_ids, start_time_idx,
                                        data, sleep_stages_origin, save_spindle_loc, sel_id_name=sel_id_name,
            Fs=Fs, epoch_length=epoch_length, window_time=epoch_length, s_loc=0, e_loc=1,
            sw_columns_ordered=sw_columns_ordered,sp_columns_ordered=sp_columns_ordered,
            pred_slow_waves=pred_slow_waves, pred_spindles =pred_spindles)
else:
    # --------------------------------------------------------------------------
    # to obtain the continious segmentations for the loose lead detection
    # 
    # whether save the continious segments or not
    # 
    # --------------------------------------------------------------------------
    save_cont_seg=False
    cont_EEG_segments_use =  save_cont_segs(in_name_temp, loading_dir_pre.out_loc_NREM_REM, sel_ids, start_time_idx, Fs,
                                            sel_id_name='good_ids', epoch_length=epoch_length,
                                            save_cont_seg=save_cont_seg)

    start_keys = list(cont_EEG_segments_use.keys())
    cont_EEG_segments_np = continious_seg_to_np(cont_EEG_segments_use, start_keys)


Only use the good_ids (annoated as "normal" by preprocessing).
Then only the good_ids' signal and sleep stages are used for further analysis.


In [ ]:
if normal_only:
    good_ids = np.where(epoch_status=='normal')[0]
    sleep_stages_orgin = deepcopy(sleep_stages)
    sleep_stages = sleep_stages[good_ids]

    epochs = epochs[good_ids]
    try:
        epoch_start_idx = epoch_start_idx_o[good_ids]
    except:
        epoch_start_idx =  np.array(epoch_start_idx_o)[good_ids.astype(int)]

Save the overall exatracted sleep-stages (raw).
Note: not only the good_ids' sleep-stages 

In [ ]:
# --------------------------------------------------------------------------
# saving the preprocessed sleep-stages
# --------------------------------------------------------------------------
if sleep_stage_preprocess_origin:
    np.save(loading_dir_pre.out_loc_NREM_REM+ in_name_temp + '_sleep_stages_orgin',sleep_stages_orgin)
    

## MT-spectral estimation and Correlation retrieval
Since the correlation extraction is done after the MT-extraction.

The b_val is feed as input to the fuction MT_based_correltion_calc_in_continious_segs.
If we do the Fisher-based standardisation (with tanh) on correlations inorder to satisfy the boundry conditions,
like correlation (1 or -1), need to be adjusted with small negation or addition with b_val. 

In [ ]:
# --------------------------------------------------------------------------
#this assign values are especillay for correlation based loose lead detection
# adjust the boundry values and mapped by tanh
# z_transform=True
# --------------------------------------------------------------------------
b_val=opt_paramters.b_val

In [ ]:

# --------------------------------------------------------------------------
# as this functions is limited to 1 sec sliding through the MT-spectrum
# as the corrrelation value of the temporal information is obtained in 1 sec resolution can bve later down sampled 
# like sliding 2 sec by removing the one sample in the middle (kind of downsampling)
# 
# The correltion is calulated bween the given frquency band f_min_interst-f_max_interst
# Here we used the bandpassed range to calculate the correltaion
#
# save_MT_spectrum: will onkygive the spectrum output which can be later used to save
# --------------------------------------------------------------------------

# --------------------------------------------------------------------------
# either filterd EEG can be fed through the MT-estimation 
# --------------------------------------------------------------------------
if filtered_EEG_MT:
    EEG_MT_feed = EEG_root_copy
else:
    EEG_MT_feed =   EEG_root
        
# --------------------------------------------------------------------------
# to calculate correlation in the db scale or given scale
# --------------------------------------------------------------------------
db_scale=False
correlation_flatten_MT_not_opt, sleep_stages_annot_flatten, MT_spec_not_db = \
                                        MT_based_correltion_calc_in_continious_segs(EEG_MT_feed, sleep_stages, cont_EEG_segments_np, 
                                        Fs,ch_names, start_time_idx=start_time_idx, T=T, db_scale=db_scale,
                                        f_min_interst=bandpass_freq[0], f_max_interst=bandpass_freq[1], 
                                        window_time=epoch_length, padding=0, sleep_stage_anoot_ext=True,
                                        interested_channels_index=[],root_data=True,
                                        b_val=b_val,inter_mediate_transform=False, 
                                        optim_bandwidth=False,
                                        save_MT_spectrum=True)
    

## Spindle and MT-coorelatiion one hot encoding
Since the spindle information must be incopertaed with the extracted MT-spectrum (and their corresponding obtained correlation).
In case if we need to exclude/ include the spindle based analysis later. Such that inorder to connect them together the one hot encoding is used, here the spindles locations are marked with one-hot encoding, and their properties (information) is passed with another variable.

In [ ]:
# --------------------------------------------------------------------------
# spindle_interest_channel= like if we are only targetting the central channels for the spindle calculation
# --------------------------------------------------------------------------
if avoid_spindle_loc:
    # sp_sw_dic_format_copy =deepcopy(sp_sw_dic_format)
    spindle_possible_channels =[]#probabily on C3 like   

    spindle_enc, spindle_info_hold =  convert_to_full_one_hot_mapping_based_cont_seg_sp_sw_all_channels(sp_sw_dic_format,
                                        sleep_stages_or,cont_EEG_segments_np,Fs, MT_spec_not_db, start_time_idx, ep_length=30,
                                        o_p_adjuster=3, boundry_cond_sec_frac=0.5,combine_intersted_channels_to_all=False,
                                        ch_names = ch_names, spindle_interest_channel=spindle_possible_channels)
    
else:
    spindle_enc=[]

if len(spindle_enc)>0:
    in_name_temp = in_name_temp+'_avoid_spindle'

## Assigning the hyper-paramters for correlation based loose-lead detection

We have the choice to use the distribution or moving window.
Any of these methods need the hyper-parameter assignment, for retrive the detect the  outlier segments
 (possible potential loose-leads segments).

### Hyper parameters for only Distribution based 
tail_check_bin, GMM_based,    factor_check, threh_prob, outlier_basic_con
### Hyper parameters for only Moving window based 
 sep, global_check, only_good_points, sorted_median_cont_grp_comp_sort_median_cond, sorted_median_cont_grp_comp_sort_max_cond, sorted_median_cont_grp_comp_sort_quan_cond 


Since the ``methodology_related_paramaters_for_outlier_detection()'' function retrive all the hyperparamters for both methods (like distribution, moving window). The user can safely ignore the unwanted hyper-parameters.




In [ ]:
tail_check_bin, GMM_based,    factor_check, threh_prob, outlier_basic_con,\
            moving_window_based, moving_window_size, th_fact,\
            sep, global_check, only_good_points,\
            sorted_median_cont_grp_comp_sort_median_cond, sorted_median_cont_grp_comp_sort_max_cond, sorted_median_cont_grp_comp_sort_quan_cond,\
            cont_seg_wise, cont_threh, threh_prob_artf_cont_seg, \
            with_conv,thresh_min_conv,thresh_in_sec,outlier_presene_con_lenth_th, \
            with_fill_period=opt_paramters.methodology_related_paramaters_for_outlier_detection()